In [55]:
import pandas as pd

subreddit_file = open('archive/50_subreddits_list.csv', mode='r', encoding='utf-8-sig')
subreddits = {}
for line in subreddit_file.readlines():
  subreddit = line.rstrip("\n").lower()
  df = pd.read_csv(f'archive/{subreddit}.csv')
  subreddits[subreddit] = df

### Get Title and Text from DataFrames
Our classifier only works based on Title and Text

In [54]:
subreddit_bodies = {}
for subreddit in subreddits:
  df = subreddits[subreddit]
  df['title'] = df['title'].fillna('')
  df['body'] = df['body'].fillna('')
  subreddit_bodies[subreddit] = df['title'] + " " + df['body'] 

subreddit_bodies['anime']

0      'Dragon Ball' Creator Akira Toryiyama Has Pass...
1        Kaguya-sama: Love Is War - Season 3 announced! 
2                         Aqua in yoga pants | Konosuba 
3                     This is not a Cigarette [Gintama] 
4         The Devil is a Part-Timer Season 2 Announced! 
                             ...                        
991    Mob Psycho 100 Season 2 - Episode 5 discussion...
992    Never thought in a million years I’d come acro...
993    I seriously hate Sundays [Engaged to the Unide...
994                "Spy Classroom" New Character Visual 
995    Hayasaka Ai in Spy Suit from "Kaguya: Love is ...
Length: 996, dtype: object

### Remove Links
Our dataset includes links of the form "\[link text](link)". Since we want to analayze the text content of the subreddits, we should filter out these links and just keep "link text"

In [3]:
import re # Regex Library

no_links = {}
for subreddit in subreddit_bodies:
  data = subreddit_bodies[subreddit]
  no_links[subreddit] = data.map(lambda txt: re.sub("\[([^\]]*)\]\(([^\)]*)\)", " \g<1> ", txt))

no_links['travel']

0      I visited North Korea recently, these are some...
1      Taken with a phone out of my hotel window in V...
2      Taking a ride on the Bernina Express through t...
3      Wife and I hate big social events and love tra...
4      The exact moment I took a step too close to th...
                             ...                        
992    Sisteron- France. Beautiful place we had a cof...
993    Croatia, probably the most beautiful country i...
994    Michelangelo's David is great, but pieta is on...
995    If you don't mind a little dust and grit and y...
996    I’d never realized how beautiful Montenegro wa...
Length: 997, dtype: object

### Tokenizing

Since reddit posts contain long sections of prose that are unique to each user's writing we felt finding attributes about the text as a whole would be difficult. Rather, we tokenize each head and body in hopes that individual words will vary between subreddits. We also turned each word to lowercase to remove issues with capitalization between different posts, since capitalization likely doesn't affect the semantic meaning of each word in a post.

Additionally, we found the special unicode character ’ (as opposed to ') in several entries across datasets likely due to differences in keyboards among different languages. We replaced the former with the latter to correctly match words with apostrophes (e.g. I’d => I'd)

In [4]:
import nltk

tokenizer = nltk.RegexpTokenizer(
  pattern=r"[\w']+", # Only match words as tokens (coarsely, \w + apostrophes)
  gaps=False,
  discard_empty=True # Remove empty tokens caused by markdown content
)
tokenized_subreddits = {}
for subreddit in no_links:
  data = no_links[subreddit]
  data = data.map(lambda txt: txt.replace("’", "'").lower())
  tokenized_subreddits[subreddit] = data.map(lambda txt: tokenizer.tokenize(txt)) 

tokenized_subreddits['history']

0      [new, discovery, mode, turns, video, game, ass...
1      [we, are, not, here, to, help, you, with, your...
2      [a, 1776, excerpt, from, john, adam's, diary, ...
3      [famous, viking, warrior, burial, revealed, to...
4      [3, 000, year, old, underwater, castle, discov...
                             ...                        
987    [dna, study, has, now, provided, support, for,...
988    [stonehenge, megalith, came, from, scotland, n...
989    [french, resistance, man, breaks, silence, ove...
990    [holy, grail, of, shipwrecks', to, be, raised,...
991    [emily, wilson's, new, translation, of, the, i...
Length: 992, dtype: object

In [56]:
# bring tokens back together as data
subreddit_classes = {}
subreddit_df = pd.DataFrame(columns=['text', 'subreddit'])

next_class = 1
for subreddit in tokenized_subreddits:
  subreddit_classes[subreddit] = next_class
  data = pd.DataFrame()
  data['text'] = tokenized_subreddits[subreddit].map(lambda arr: ' '.join(arr))
  data['subreddit'] = next_class
  subreddit_df = pd.concat([subreddit_df, data], ignore_index=True)
  next_class += 1

subreddit_df

,text,subreddit
0,my cab driver tonight was so excited to share ...,1
1,guardians of the front page,1
2,gas station worker takes precautionary measure...,1
3,the conversation my son and i will have on chr...,1
4,the denver broncos have the entire town of sou...,1
...,...,...
49261,how come no one has invented a foot pedal for ...,50
49262,why are trans people talked about so much desp...,50
49263,did your penis ever fall asleep like your legs...,50
49264,someone stole my bike i tracked it to its loca...,50


In [72]:
from sklearn.model_selection import train_test_split

X = subreddit_df['text']
y = subreddit_df['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
X_train = X_train.astype('str').to_numpy()
X_test = X_test.astype('str')
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [73]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

reddit_classifer = Pipeline([
  ('cv', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('sgd', SGDClassifier()),
])

In [74]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(reddit_classifer, {
  'cv__ngram_range': [(1, 1), (1, 2)],
  'cv__max_df': [0.8, 0.9, 0.95, 1.0],
  'cv__min_df': [1, 3, 5],
  'cv__max_features': [1000, 2000, None],
  'sgd__loss': ['hinge', 'log_loss']
}, n_jobs=8, verbose=3)
grid_search.fit(X_train[:10000], y_train[:10000])
print(grid_search.best_score_)
grid_search.best_params_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 2/5] END cv__max_df=0.8, cv__max_features=1000, cv__min_df=1, cv__ngram_range=(1, 1), sgd__loss=hinge;, score=0.449 total time=   2.2s
[CV 1/5] END cv__max_df=0.8, cv__max_features=1000, cv__min_df=1, cv__ngram_range=(1, 1), sgd__loss=hinge;, score=0.447 total time=   2.3s
[CV 4/5] END cv__max_df=0.8, cv__max_features=1000, cv__min_df=1, cv__ngram_range=(1, 1), sgd__loss=hinge;, score=0.451 total time=   2.3s
[CV 5/5] END cv__max_df=0.8, cv__max_features=1000, cv__min_df=1, cv__ngram_range=(1, 1), sgd__loss=hinge;, score=0.451 total time=   2.3s
[CV 3/5] END cv__max_df=0.8, cv__max_features=1000, cv__min_df=1, cv__ngram_range=(1, 1), sgd__loss=hinge;, score=0.444 total time=   2.3s
[CV 1/5] END cv__max_df=0.8, cv__max_features=1000, cv__min_df=1, cv__ngram_range=(1, 1), sgd__loss=log_loss;, score=0.469 total time=   2.7s
[CV 2/5] END cv__max_df=0.8, cv__max_features=1000, cv__min_df=1, cv__ngram_range=(1, 1), sgd__loss=

KeyboardInterrupt: 

In [75]:
reddit_classifer = Pipeline([
  ('cv', CountVectorizer(ngram_range=(1,2), lowercase=False)),
  ('tfidf', TfidfTransformer()),
  ('sgd', SGDClassifier()),
])
reddit_classifer.fit(X_train, y_train)

Pipeline(steps=[('cv', CountVectorizer(lowercase=False, ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()), ('sgd', SGDClassifier())])

In [76]:
import numpy as np

y_pred = reddit_classifer.predict(X_test)
np.mean(y_pred == y_test)

0.6345701063570675

In [ ]:
def predictPrompt(prompt):
  predicted_class = reddit_classifer.predict([prompt])[0].item()
  for subreddit in subreddit_classes:
    if subreddit_classes[subreddit] == predicted_class:
      return subreddit
  print("No applicable subreddit (should be unreachable)")

predictPrompt("How to make a cake from scratch")

'diy'

In [87]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/predict")
def predictBackend():
  print(request.args.get('prompt'))
  prediction = predictPrompt(request.args.get('prompt'))
  return jsonify({'prediction': prediction})

app.run(host='0.0.0.0', port=3000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3000
 * Running on http://192.168.1.86:3000
Press CTRL+C to quit


rock rock i love rock


127.0.0.1 - - [28/Nov/2024 14:07:28] "GET /predict?prompt=rock%20rock%20i%20love%20rock HTTP/1.1" 200 -


rock


...

127.0.0.1 - - [28/Nov/2024 14:12:47] "GET /predict?prompt=rock HTTP/1.1" 200 -


rock


127.0.0.1 - - [28/Nov/2024 14:13:39] "GET /predict?prompt=rock HTTP/1.1" 200 -


my girlfriend broke up with me what should i do?


127.0.0.1 - - [28/Nov/2024 14:14:02] "GET /predict?prompt=my%20girlfriend%20broke%20up%20with%20me%20what%20should%20i%20do? HTTP/1.1" 200 -
